## 06. 군집화 실습 - 고객 세그먼테이션
### 고객 세그먼테이션의 정의와 기법
1. 정의 : 다양한 기준으로 고객을 분류하는 기법을 지칭
2. 활용 : CRM(Customer Relation Mangement, 고객 관계 관리)이나 마케팅의 중요 기반 요소
3. 목표 : 타겟 마케팅 - 고객 특성에 맞게 세분화해서 그 유형에 따라서 맞춤형 마케팅이나 서비스를 제공
4. 예제 : RFM 기법 사용 - Recency(R), Frequency(F), Monetary Value(M)
- Recency : 가장 최근 상품 구입 일에서 오늘까지의 시간
- Frequency : 상품 구매 횟수
- Monetary Value : 총 구매 금액

### 데이터 세트 로딩과 데이터 클렌징
- 데이터 세트 [다운로드](https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx)

#### 참고
- Online Retail Data Set [link](https://archive.ics.uci.edu/ml/datasets/online+retail)